# aggregate_results_of_different_types.py - ELI5 Walkthrough
This notebook recreates `python/samples/getting_started/workflows/parallelism/aggregate_results_of_different_types.py` so you can follow the fan-out/fan-in pattern step by step.


## Big Picture
A dispatcher fans out a list of numbers to two workers (Average and Sum). Their outputs are fanned back in by an Aggregator that collects multiple result types in a single list.


## Key Ingredients
- `WorkflowBuilder.add_fan_out_edges` and `.add_fan_in_edges` express parallel branches and joins.
- Executors send messages with typed `WorkflowContext`.
- Streaming run emits `WorkflowOutputEvent` when the fan-in node yields results.


### Workflow Diagram
```mermaid
flowchart LR
    Start(["Number List"]) --> Dispatcher[[Dispatcher]]
    Dispatcher --> Average[[Average]]
    Dispatcher --> Sum[[Sum]]
    Average --> Aggregator[[Aggregator]]
    Sum --> Aggregator
    Aggregator --> Output(["[sum, average]"])
```


### Step 1: Imports and scenario overview
We load the Agent Framework primitives, configure environment variables, and keep the docstring describing the fan-out/fan-in demo.


In [ ]:
# Copyright (c) Microsoft. All rights reserved.
from dotenv import load_dotenv
load_dotenv()

import asyncio
import random

from agent_framework import Executor, WorkflowBuilder, WorkflowContext, WorkflowOutputEvent, handler
from typing_extensions import Never

"""
Sample: Concurrent fan out and fan in with two different tasks that output results of different types.

Purpose:
Show how to construct a parallel branch pattern in workflows. Demonstrate:
- Fan out by targeting multiple executors from one dispatcher.
- Fan in by collecting a list of results from the executors.
- Simple tracing using AgentRunEvent to observe execution order and progress.

Prerequisites:
- Familiarity with WorkflowBuilder, executors, edges, events, and streaming runs.
"""




### Step 2: Dispatcher fans out the input
`Dispatcher` validates the incoming list and forwards it to each parallel worker.


In [ ]:
class Dispatcher(Executor):
    """
    The sole purpose of this decorator is to dispatch the input of the workflow to
    other executors.
    """

    @handler
    async def handle(self, numbers: list[int], ctx: WorkflowContext[list[int]]):
        if not numbers:
            raise RuntimeError("Input must be a valid list of integers.")

        await ctx.send_message(numbers)




### Step 3: Worker executors compute their metrics
`Average` and `Sum` each take the same list and emit their numeric result.


In [ ]:
class Average(Executor):
    """Calculate the average of a list of integers."""

    @handler
    async def handle(self, numbers: list[int], ctx: WorkflowContext[float]):
        average: float = sum(numbers) / len(numbers)
        await ctx.send_message(average)




In [ ]:
class Sum(Executor):
    """Calculate the sum of a list of integers."""

    @handler
    async def handle(self, numbers: list[int], ctx: WorkflowContext[int]):
        total: int = sum(numbers)
        await ctx.send_message(total)




### Step 4: Aggregator fans the results back in
`Aggregator` receives a heterogeneous list (`int | float`) and yields it as the final workflow output.


In [ ]:
class Aggregator(Executor):
    """Aggregate the results from the different tasks and yield the final output."""

    @handler
    async def handle(self, results: list[int | float], ctx: WorkflowContext[Never, list[int | float]]):
        """Receive the results from the source executors.

        The framework will automatically collect messages from the source executors
        and deliver them as a list.

        Args:
            results (list[int | float]): execution results from upstream executors.
                The type annotation must be a list of union types that the upstream
                executors will produce.
            ctx (WorkflowContext[Never, list[int | float]]): A workflow context that can yield the final output.
        """
        await ctx.yield_output(results)




### Step 5: Wire and run the workflow
`main()` constructs the graph, executes it with random numbers, and prints the collected outputs.


In [ ]:
async def main() -> None:
    # 1) Create the executors
    dispatcher = Dispatcher(id="dispatcher")
    average = Average(id="average")
    summation = Sum(id="summation")
    aggregator = Aggregator(id="aggregator")

    # 2) Build a simple fan out and fan in workflow
    workflow = (
        WorkflowBuilder()
        .set_start_executor(dispatcher)
        .add_fan_out_edges(dispatcher, [average, summation])
        .add_fan_in_edges([average, summation], aggregator)
        .build()
    )

    # 3) Run the workflow
    output: list[int | float] | None = None
    async for event in workflow.run_stream([random.randint(1, 100) for _ in range(10)]):
        if isinstance(event, WorkflowOutputEvent):
            output = event.data

    if output is not None:
        print(output)




### Step 6: Try it yourself
Use the helper below. In notebooks it awaits `main()` on the active loop; in scripts it falls back to `asyncio.run(main())`.


In [ ]:
import asyncio

# Helper for notebooks vs. scripts
loop = asyncio.get_event_loop()
if loop.is_running():
    # Jupyter/VS Code notebooks already have an event loop, so await directly.
    await main()
else:
    asyncio.run(main())
